In [1]:
! pip3 install tensorflow tensorflow-hub numpy 
import os
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import re
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType, connections, utility

2023-05-23 17:05:18.090314: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-23 17:05:18.116175: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-23 17:05:18.116756: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 17:05:18.713497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Download the model and load the model

module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)

In [3]:
# Function to generate embeddings

def embeddings(text):
    return np.array(model(text)).flatten().tolist()

In [4]:
# Dataset available at https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
# Download the dataset and extract the files before running this cell
# We will create the embeddings from this dataset using the universal-sentence-encoder model

file_path = 'SMSSpamCollection'

with open(file_path) as file:
    lines = [line for line in file]

msgs = [x.split('\t')[1].replace('\n', '')   for x in lines]
embdngs = [embeddings([x]) for x in msgs]
indx = list(range(1, len(msgs)+1))

data_to_insert = [indx, msgs, embdngs]

2023-05-23 17:09:46.756712: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string
	 [[{{node inputs}}]]


In [5]:
# Connect to milvus database

connections.connect(
  alias="default",
  host='localhost',
  port='19530'
)

In [6]:
# Field Schema
id = FieldSchema(
  name="id",
  dtype=DataType.INT64,
  is_primary=True,
)
message = FieldSchema(
  name="message",
  dtype=DataType.VARCHAR,
  max_length=6000,
)
message_vec = FieldSchema(
  name="message_embeddings",
  dtype=DataType.FLOAT_VECTOR,
  dim=512
)
# collection schema
collection_schema = CollectionSchema(
  fields=[id, message, message_vec],
  description="Spam SMS collection"
)
# Create collection
collection = Collection(
    name="Spam_Test",
    schema=collection_schema,
    using='default')
utility.list_collections()

['Album1', 'Spam_Test']

In [7]:
# Insert entities

data_insert = collection.insert(data_to_insert)

In [8]:
# Create Index
index_params = {
  "metric_type":"L2",
  "index_type":"IVF_FLAT",
  "params":{"nlist":1024},
  "index_name": "SMS_IVF_FLAT_TEST"
}

# Index on vector field
collection.create_index(
  field_name="message_embeddings", 
  index_params=index_params
)

Status(code=0, message=)

In [9]:
# Load the collection

collection.load(replica_number=1)

In [10]:
# test message
test_message = ["claim prize"]
test_message_vector = embeddings(test_message)

In [11]:
## Vector Similarity Search
search_params = {"metric_type": "L2", "params": {"nprobe": 64}}

results = collection.search(
	data=[test_message_vector], 
	anns_field="message_embeddings", 
	param=search_params,
	limit=5, 
	expr=None,
	output_fields=['message']
)

for result in results[0]:
    print (result)

id: 4048, distance: 1.1847829818725586, entity: {'message': 'Win a £1000 cash prize or a prize worth £5000'}
id: 577, distance: 1.1955623626708984, entity: {'message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327'}
id: 9, distance: 1.2261712551116943, entity: {'message': 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'}
id: 4282, distance: 1.2261712551116943, entity: {'message': 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'}
id: 4037, distance: 1.226269245147705, entity: {'message': 'YOU ARE CHOSEN TO RECEIVE A £350 AWARD! Pls call claim number 09066364311 to collect your award which you are selected to receive as a valued mobile customer.'}
